In [1]:
# auto reload magic
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoProcessor, AutoModel
from datasets import load_dataset
import soundfile as sf
# processor = AutoProcessor.from_pretrained("facebook/hubert-base-ls960")
# processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
# model = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [3]:
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

In [4]:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

# batch_inputs = processor(ds["speech"][0:3], return_tensors="pt", padding=True)
# hidden_states = model(**batch_inputs).last_hidden_state

c:\Users\kinet\anaconda3\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for hf-internal-testing/librispeech_asr_dummy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hf-internal-testing/librispeech_asr_dummy
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [7]:
from modeling.speech_encoder import SpeechEncoder

speech_encoder = SpeechEncoder("facebook/hubert-base-ls960", 1024)


c:\Users\kinet\anaconda3\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model 

In [8]:
x = speech_encoder(ds["speech"][0:3])

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [9]:
x[0].shape, x[1].shape

(torch.Size([3, 124, 1024]), torch.Size([3, 124]))

In [12]:
x[0]

tensor([[[ 0.2111, -0.0168,  0.0062,  ...,  0.0667,  0.0676,  0.0423],
         [ 0.0717,  0.0738,  0.0336,  ...,  0.0785,  0.0360,  0.0960],
         [ 0.0540, -0.0797, -0.0423,  ...,  0.1826,  0.1093,  0.0815],
         ...,
         [-0.1273, -0.0323,  0.0087,  ..., -0.0300,  0.1291, -0.0452],
         [-0.1273, -0.0323,  0.0087,  ..., -0.0300,  0.1291, -0.0452],
         [-0.1273, -0.0323,  0.0087,  ..., -0.0300,  0.1291, -0.0452]],

        [[ 0.2582, -0.0732, -0.0400,  ...,  0.0500,  0.0669,  0.0614],
         [ 0.0977, -0.0165, -0.0305,  ..., -0.0058,  0.0943,  0.1681],
         [ 0.0829,  0.0346, -0.0634,  ..., -0.0404,  0.0103,  0.1833],
         ...,
         [-0.1704,  0.0190,  0.0266,  ..., -0.0338,  0.0625, -0.0431],
         [-0.1704,  0.0190,  0.0266,  ..., -0.0338,  0.0625, -0.0431],
         [-0.1704,  0.0190,  0.0266,  ..., -0.0338,  0.0625, -0.0431]],

        [[ 0.2396, -0.0706, -0.0021,  ...,  0.0750,  0.0578,  0.0155],
         [ 0.0786, -0.0516, -0.0093,  ...,  0

In [ ]:
from transformers import AutoTokenizer
import transformers 
import torch
model = "TinyLlama/TinyLlama-1.1B-Chat-v0.4"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

CHAT_EOS_TOKEN_ID = 32002

prompt = "How to get in a good university?"
formatted_prompt = (
    f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
)


sequences = pipeline(
    formatted_prompt,
    do_sample=True,
    top_k=50,
    top_p = 0.9,
    num_return_sequences=1,
    repetition_penalty=1.1,
    max_new_tokens=1024,
    eos_token_id=CHAT_EOS_TOKEN_ID,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")


In [ ]:
import torch
from transformers import Speech2Text2Processor, SpeechEncoderDecoderModel
from datasets import load_dataset
import soundfile as sf

model = SpeechEncoderDecoderModel.from_pretrained("facebook/s2t-wav2vec2-large-en-de")
processor = Speech2Text2Processor.from_pretrained("facebook/s2t-wav2vec2-large-en-de")


def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch


ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

inputs = processor(ds["speech"][0], sampling_rate=16_000, return_tensors="pt")
generated_ids = model.generate(inputs=inputs["input_values"], attention_mask=inputs["attention_mask"])

transcription = processor.batch_decode(generated_ids)

In [13]:
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.4")


In [38]:
prompt = """<|im_start|>user\n"""
formatted_prompt = tokenizer(prompt, return_tensors="pt")

In [39]:
formatted_prompt

{'input_ids': tensor([[    1,  1919,  1301, 29581,   278, 10348, 29889, 32002,    13, 32001,
           465, 22137,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}